In [5]:
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from datetime import datetime
import sys
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo

# --- 1. SETUP: Direct File Path ---
input_csv_path = r"G:\My Drive\INPUTS\EXCEL\Miners\koyfin_Junior Miners.csv"

if not os.path.exists(input_csv_path):
    print(f"❌ Error: Input file not found at: {input_csv_path}")
    print(f"Please ensure the file exists at the specified location.")
    sys.exit()
else:
    print(f"✅ Input file found: {os.path.basename(input_csv_path)}")
    print(f"📅 File date: {datetime.fromtimestamp(os.path.getmtime(input_csv_path)).strftime('%Y-%m-%d %H:%M:%S')}")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_excel_name = f"Junior_Miner_Analysis_V6_{timestamp}.xlsx"
output_folder = r"G:\My Drive\OUTPUTS\EXCEL\Miner"
os.makedirs(output_folder, exist_ok=True)
output_excel_path = os.path.join(output_folder, output_excel_name)

print(f"Input file: {input_csv_path}")
print(f"Output file will be saved to: {output_excel_path}")
print(f"📅 Timestamp: {timestamp}")

# --- LOAD SELECT CSV FOR FILTERING (ISIN-BASED MATCHING) ---
select_csv_path = r"G:\My Drive\INPUTS\EXCEL\Miners\koyfin_Select Miners.csv"
select_isins = set()
select_tickers_list = []
select_count = 0

if os.path.exists(select_csv_path):
    print(f"\n✅ SELECT file found: {os.path.basename(select_csv_path)}")
    try:
        select_df = pd.read_csv(select_csv_path)
        select_df.columns = select_df.columns.str.strip()
        if 'ISIN' in select_df.columns:
            select_isins = set(select_df['ISIN'].dropna().str.strip())
            select_tickers_list = select_df['Ticker'].dropna().str.strip().tolist()
            select_count = len(select_isins)
            print(f"   📌 Loaded {select_count} companies for comparison (via ISIN matching)")
            if select_count > 0:
                sample_list = ', '.join(select_tickers_list[:5])
                print(f"   🔍 Sample: {sample_list}{'...' if select_count > 5 else ''}")
        else:
            print(f"   ⚠️ Warning: 'ISIN' column not found in SELECT file")
            print(f"   Falling back to Ticker matching...")
            if 'Ticker' in select_df.columns:
                select_isins = set(select_df['Ticker'].dropna().str.strip().str.upper())
                select_count = len(select_isins)
                print(f"   📌 Loaded {select_count} companies (via Ticker)")
            else:
                print(f"   ⚠️ Neither ISIN nor Ticker column found")
    except Exception as e:
        print(f"   ⚠️ Error reading SELECT file: {e}")
        select_isins = set()
else:
    print(f"\n⚠️ SELECT file not found: {select_csv_path}")
    print(f"   Proceeding without SELECT filtering")
    select_isins = set()

# --- HELPER FUNCTIONS ---
def human_name_clusters(profiles, factor_score_cols):
    """Assigns descriptive names to the three clusters."""
    names = {}
    remaining_ids = list(profiles.index)
    grower_id = profiles.loc[remaining_ids, 'RealGrowth_Checker_Score'].idxmax()
    names[grower_id] = "Speculative Growers"
    remaining_ids.remove(grower_id)
    if profiles.loc[remaining_ids[0], 'FinancialStability_Score'] > profiles.loc[remaining_ids[1], 'FinancialStability_Score']:
        names[remaining_ids[0]] = "Stalwart Survivors"
        names[remaining_ids[1]] = "Investing Survivors"
    else:
        names[remaining_ids[1]] = "Stalwart Survivors"
        names[remaining_ids[0]] = "Investing Survivors"
    return names

def make_pca_scatter(scaled_data, labels, output_path):
    """Create and save PCA scatter plot."""
    pca = PCA(n_components=2, random_state=42)
    pca_data = pca.fit_transform(scaled_data)
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(pca_data[:, 0], pca_data[:, 1], c=labels, alpha=0.7, cmap='viridis')
    plt.xlabel(f'PCA Component 1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
    plt.ylabel(f'PCA Component 2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
    plt.title('Junior Miners — PCA Scatter (k=3)')
    plt.colorbar(scatter, label='Cluster')
    plt.tight_layout()
    png_path = os.path.splitext(output_path)[0] + "_PCA_k3.png"
    plt.savefig(png_path, dpi=160, bbox_inches='tight')
    plt.close()
    print(f"📊 PCA plot saved to: {png_path}")
    return png_path

# --- 2. DATA LOADING AND CLEANING ---
df = pd.read_csv(input_csv_path)
df.columns = df.columns.str.strip()
print(f"\n📈 Loaded {len(df)} companies with {len(df.columns)} columns")

# --- ADD SELECTION_STATUS COLUMN (ISIN-BASED MATCHING) ---
df['ISIN'] = df['ISIN'].astype(str).str.strip()
df['Selection_Status'] = df['ISIN'].apply(
    lambda x: "SELECT" if x in select_isins else "ALL"
)
select_in_universe = len(df[df['Selection_Status'] == 'SELECT'])
if select_count > 0:
    print(f"   📊 Selection Status: {select_in_universe}/{select_count} SELECT companies found in data\n")

# --- 3. DATA SCREENING PIPELINE ---
print("🔍 Applying Data Quality Screen...")
critical_kpis = [
    'Market Cap', 'Net Debt / MKT Cap', 'cash from ops / mkt cap',
    'CapEX V Mkt CAP (3Y AV)', 'P/B (LTM)', 'Shrs % Chg (1Y)', 'Total Return (1Y)'
]
check_df = df[['Ticker', 'Name'] + critical_kpis].copy()
for col in critical_kpis:
    check_df[col] = pd.to_numeric(check_df[col], errors='coerce')

missing_data_mask = check_df[critical_kpis].isna().any(axis=1)
data_catcher_df = df[missing_data_mask].copy()
data_catcher_df['Reason_for_Exclusion'] = "Missing data in one or more critical KPIs"
analysis_df = df[~missing_data_mask].copy()
print(f"   - {len(data_catcher_df)} companies excluded for missing critical data.")

print("⚠️ Applying Collapsed Market Cap Screen...")
mkt_cap_col = 'Market Cap'
mkt_cap_3y_avg_col = 'Market Cap (3YAVG)'
analysis_df[mkt_cap_col] = pd.to_numeric(analysis_df[mkt_cap_col], errors='coerce')
analysis_df[mkt_cap_3y_avg_col] = pd.to_numeric(analysis_df[mkt_cap_3y_avg_col], errors='coerce')
denominator = analysis_df[mkt_cap_3y_avg_col].replace(0, np.nan)
analysis_df['Mkt_Cap_Collapse_Ratio'] = analysis_df[mkt_cap_col] / denominator
collapse_threshold = 0.75
collapse_mask = analysis_df['Mkt_Cap_Collapse_Ratio'] < collapse_threshold
collapsed_cap_df = analysis_df[collapse_mask].copy()
collapsed_cap_df['Reason_for_Exclusion'] = f"Market Cap is less than {collapse_threshold:.0%} of its 3Y Average"
analysis_df = analysis_df[~collapse_mask].copy()
print(f"   - {len(collapsed_cap_df)} companies flagged for collapsed market cap.")
print(f"   ✅ {len(analysis_df)} companies remaining for full analysis.")

if analysis_df.empty:
    print("\n❌ ERROR: No companies remained after all screening steps!")
    sys.exit()

# --- 4. FEATURE ENGINEERING & DATA TRANSFORMATION ---
print("\n🛠️ Engineering and transforming features for scoring consistency...")
analysis_df['Abs_Capex_LTM'] = analysis_df['Capital Expenditure (LTM)'].abs()
epsilon = 1e-6 
analysis_df['Cash_to_Capex_Coverage'] = analysis_df['Cash/ST Investments (LTM)'] / (analysis_df['Abs_Capex_LTM'] + epsilon)

metrics_to_invert = [
    'Net Debt / MKT Cap', 'Head Office Effeciancy', 'Shrs % Chg (1Y)', 'P/B (LTM)'
]
for metric in metrics_to_invert:
    new_col_name = f"Inv_{metric.replace(' ', '_').replace('/', '_').replace('%','').replace('(','').replace(')','')}"
    analysis_df[new_col_name] = analysis_df[metric] * -1

# --- 5. MODEL DEFINITION ---
factor_kpis = {
    'Survival': {'Burn in Months': {'scoring': 'custom_runway'}, 'Inv_Net_Debt___MKT_Cap': {'scoring': 'higher_is_better'}, 'Cash_to_Capex_Coverage': {'scoring': 'higher_is_better'}},
    'InvestmentQuality': {'CapEX V Mkt CAP (3Y AV)': {'scoring': 'higher_is_better'},'Inv_Head_Office_Effeciancy': {'scoring': 'higher_is_better'}},
    'FinancialStability': {'cash from ops / mkt cap': {'scoring': 'higher_is_better'},'Inv_Net_Debt___MKT_Cap': {'scoring': 'higher_is_better'}},
    'ShareholderFriendliness': {'Inv_Shrs__Chg_1Y': {'scoring': 'higher_is_better'}},
    'Valuation': {'Inv_P_B_LTM': {'scoring': 'higher_is_better'}},
    'MarketMomentum': {'Total Return (1Y)': {'scoring': 'higher_is_better'}},
    'RealGrowth_Checker': {'Net Property/Plant and Equip./CAGR (3Y FY)': {'scoring': 'higher_is_better'},'Total Assets/CAGR (3Y FY)': {'scoring': 'higher_is_better'},'CAPEX/CAGR (3Y TTM)': {'scoring': 'higher_is_better'}}
}
print(f"\n⚙️ Scoring Model: {len(factor_kpis)} factors defined.")

# --- 6. SCORING ENGINE ---
def score_runway_kpi(series):
    scores = pd.Series(np.nan, index=series.index)
    scores[series >= 0] = 5
    risky_group = series[series < 0]
    if not risky_group.empty:
        try:
            risky_scores = pd.qcut(risky_group.abs(), 4, labels=False, duplicates='drop') + 1
            scores.loc[risky_group.index] = risky_scores
        except ValueError:
            scores.loc[risky_group.index] = 2
    scores.fillna(3, inplace=True)
    return scores.astype(int)

score_df = analysis_df.copy()
all_kpis = sorted(list(set(kpi for factor in factor_kpis.values() for kpi in factor)))
print(f"🔢 Processing {len(all_kpis)} KPIs for scoring...")

for col in all_kpis:
    if col in score_df.columns:
        score_df[col] = pd.to_numeric(score_df[col], errors='coerce')
        median_val = score_df[col].median()
        score_df[col].fillna(median_val, inplace=True)

kpis_to_score = [k for k in all_kpis if k != 'Inv_P_B_LTM']
for kpi in kpis_to_score:
    score_col_name = f'{kpi}_Score'
    scoring_type = next(factor[kpi]['scoring'] for factor in factor_kpis.values() if kpi in factor)
    try:
        if scoring_type == 'custom_runway':
            score_df[score_col_name] = score_runway_kpi(score_df[kpi])
        else:
            score_df[score_col_name] = pd.qcut(score_df[kpi], 5, labels=False, duplicates='drop') + 1
    except ValueError:
        score_df[score_col_name] = 3
    score_df[score_col_name].fillna(3, inplace=True)

score_df['Stage_Proxy'] = pd.qcut(score_df['cash from ops / mkt cap'], 3, labels=['Explorer', 'Developer', 'Producer'], duplicates='drop')
def safe_qcut_scorer_inv_pb(group):
    try:
        group['Inv_P_B_LTM_Score'] = pd.qcut(group['Inv_P_B_LTM'], 5, labels=False, duplicates='drop') + 1
    except ValueError:
        group['Inv_P_B_LTM_Score'] = 3
    return group
score_df = score_df.groupby('Stage_Proxy', group_keys=False).apply(safe_qcut_scorer_inv_pb)
score_df['Inv_P_B_LTM_Score'].fillna(3, inplace=True)

# --- 7. FINAL CALCULATION & NORMALIZATION ---
for factor, kpis in factor_kpis.items():
    score_df[f'{factor}_Score_Raw'] = sum(score_df[f'{kpi}_Score'] for kpi in kpis)

print("\n⚖️ Normalizing factor scores and calculating Total Scores...")
for factor, kpis in factor_kpis.items():
    raw_col_name = f'{factor}_Score_Raw'
    norm_col_name = f'{factor}_Score'
    min_score = len(kpis) * 1
    max_score = len(kpis) * 5
    if (max_score - min_score) > 0:
        score_df[norm_col_name] = (score_df[raw_col_name] - min_score) / (max_score - min_score)
    else:
        score_df[norm_col_name] = (score_df[raw_col_name] - 1) / 4.0

factor_score_cols = [f'{f}_Score' for f in factor_kpis.keys()]
score_df['Total_Score_Additive'] = score_df[factor_score_cols].sum(axis=1)
score_df['Total_Score_Multiplicative'] = score_df[factor_score_cols].prod(axis=1)
score_df['Rank_Additive'] = score_df['Total_Score_Additive'].rank(method='min', ascending=False).astype(int)
score_df['Rank_Multiplicative'] = score_df['Total_Score_Multiplicative'].rank(method='min', ascending=False).astype(int)
total_companies = len(score_df)
conditions = [score_df['Rank_Additive'] <= total_companies * 0.2, score_df['Rank_Additive'] <= total_companies * 0.6]
choices = ['Tier 1: Investigate', 'Tier 2: Monitor']
score_df['Tier'] = np.select(conditions, choices, default='Tier 3: Caution')
print(f"✅ Scoring & Normalization complete for {len(score_df)} companies")

# --- 8. K-MEANS CLUSTERING ANALYSIS ---
print(f"\n🎯 Performing K-Means Clustering (k=3)...")
cluster_data = score_df[factor_score_cols]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(cluster_data)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(scaled_data)
score_df['Cluster'] = cluster_labels
profiles = score_df.groupby('Cluster')[factor_score_cols].mean()
cluster_names = human_name_clusters(profiles, factor_score_cols)
score_df['Cluster_Name'] = score_df['Cluster'].map(cluster_names)
sil_score = silhouette_score(scaled_data, cluster_labels)
print(f"📊 Clustering Results:")
print(f"   🎯 Silhouette Score: {sil_score:.3f} ({'Good' if sil_score > 0.5 else 'Fair' if sil_score > 0.25 else 'Poor'})")
for cluster_id, name in sorted(cluster_names.items()):
    count = len(score_df[score_df['Cluster'] == cluster_id])
    print(f"       {cluster_id}: {name} ({count} companies)")
png_path = make_pca_scatter(scaled_data, cluster_labels, output_excel_path)

# --- 9. TIER & CLUSTER PROFILE ANALYSIS ---
print("\n📝 Generating Tier & Cluster Profile Analysis tables...")
original_kpis_to_profile = [
    'Burn in Months','Net Debt / MKT Cap','CapEX V Mkt CAP (3Y AV)','Head Office Effeciancy',
    'cash from ops / mkt cap','Shrs % Chg (1Y)','P/B (LTM)','Total Return (1Y)', 'Total Return (3M)',
    'Net Property/Plant and Equip./CAGR (3Y FY)','Total Assets/CAGR (3Y FY)','CAPEX/CAGR (3Y TTM)',
    'Cash_to_Capex_Coverage'
]
score_df['Total Return (3M)'] = pd.to_numeric(score_df['Total Return (3M)'], errors='coerce')

percentiles_to_calc = [0.25, 0.50, 0.75]
percentile_labels = ['25th Percentile', 'Median (50%)', '75th Percentile']

def create_profile_table(df, group_col):
    all_tables = []
    if group_col in df.columns:
        for group_name in sorted(df[group_col].unique()):
            group_df = df[df[group_col] == group_name]
            if not group_df.empty:
                percentile_table = group_df[original_kpis_to_profile].quantile(percentiles_to_calc).reset_index(drop=True)
                percentile_table.index = percentile_labels
                percentile_table.reset_index(inplace=True)
                percentile_table.rename(columns={'index': 'Percentile'}, inplace=True)
                percentile_table[group_col] = group_name
                all_tables.append(percentile_table)
        
        if all_tables:
            summary_df = pd.concat(all_tables, ignore_index=True)
            cols_to_order = [group_col, 'Percentile'] + [kpi for kpi in original_kpis_to_profile if kpi in summary_df.columns]
            return summary_df[cols_to_order]
    return pd.DataFrame()

tier_percentile_summary_df = create_profile_table(score_df, 'Tier')
cluster_percentile_summary_df = create_profile_table(score_df, 'Cluster_Name')

# --- 10. COMBINED TIER-CLUSTER 3M RETURN ANALYSIS ---
print("📈 Generating Combined Tier-Cluster 3M Return Analysis...")
score_df['Tier_Cluster_Combo'] = score_df['Tier'] + ' - ' + score_df['Cluster_Name']
combo_return_analysis = score_df.groupby('Tier_Cluster_Combo')['Total Return (3M)'].agg(
    Company_Count='size',
    **{f'{int(p*100)}th_Percentile': lambda x, p=p: x.quantile(p) for p in percentiles_to_calc}
).reset_index()
combo_return_analysis.rename(columns={'50th_Percentile': 'Median_Return_3M'}, inplace=True)

# --- 11. OUTPUT GENERATION WITH TIMESTAMP IN METADATA ---
final_cols_order = [
    'Rank_Additive', 'Rank_Multiplicative', 'Ticker', 'Name', 'Selection_Status',
    'Total_Score_Additive', 'Total_Score_Multiplicative', 'Tier', 'Cluster_Name', 'Stage_Proxy'
] + factor_score_cols
full_results_df = score_df[final_cols_order].sort_values('Rank_Additive')

full_results_df['Analysis_Date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print(f"\n📊 Generating Excel Output...")
overview_data = pd.DataFrame({
    "Metric": ["Analysis Date", "Input Rows", "Analyzed Rows (Post-Screen)", "Data Catcher Rows", "Collapsed Cap Rows", "k (clusters)", "Silhouette Score"],
    "Value": [datetime.now().strftime("%Y-%m-%d %H:%M:%S"), len(df), len(analysis_df), len(data_catcher_df), len(collapsed_cap_df), 3, round(sil_score, 3)]
})
cluster_summary = profiles.copy()
cluster_summary["Count"] = score_df.groupby("Cluster").size()
cluster_summary["Cluster_Name"] = cluster_summary.index.map(cluster_names)
cluster_summary = cluster_summary.reset_index(drop=True).reindex(columns=["Cluster_Name", "Count"] + factor_score_cols)

def clean_sheet_name(name):
    for ch in ['\\', '/', '*', '?', '[', ']', ':', '(', ')']:
        name = name.replace(ch, '-')
    return name[:31]

cluster_sheet_names = {cluster_id: clean_sheet_name(name) for cluster_id, name in cluster_names.items()}

with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
    overview_data.to_excel(writer, sheet_name='Overview', index=False)
    cluster_summary.to_excel(writer, sheet_name='Cluster Profiles', index=False)
    full_results_df.to_excel(writer, sheet_name='Full Scoring & Clusters', index=False)
    
    if not data_catcher_df.empty:
        data_catcher_df.to_excel(writer, sheet_name='Data Catcher', index=False)
    
    if not collapsed_cap_df.empty:
        collapsed_cap_df.to_excel(writer, sheet_name='Collapsed Mkt Cap Flag', index=False)
        
    for cluster_id, name in cluster_names.items():
        cluster_df = full_results_df[full_results_df['Cluster_Name'] == name].copy()
        cluster_df_top = cluster_df.sort_values('Rank_Additive').head(50)
        sheet_name = cluster_sheet_names[cluster_id]
        cluster_df_top.to_excel(writer, sheet_name=sheet_name, index=False)
    
    for tier_name in ['Tier 1: Investigate', 'Tier 2: Monitor', 'Tier 3: Caution']:
        tier_df = full_results_df[full_results_df['Tier'] == tier_name]
        sheet_name = tier_name.replace(":", "")
        tier_df.to_excel(writer, sheet_name=sheet_name, index=False)
        
    if not tier_percentile_summary_df.empty:
        tier_percentile_summary_df.to_excel(writer, sheet_name='Tier Profile Analysis', index=False, header=True)
    if not cluster_percentile_summary_df.empty:
        cluster_percentile_summary_df.to_excel(writer, sheet_name='Cluster Profile Analysis', index=False, header=True)
    if not combo_return_analysis.empty:
        combo_return_analysis.to_excel(writer, sheet_name='Tier-Cluster 3M Returns', index=False, header=True)

# --- CREATE EXCEL TABLE FOR FULL SCORING TAB ---
try:
    wb = load_workbook(output_excel_path)
    ws = wb['Full Scoring & Clusters']
    
    table_ref = f"A1:{chr(64 + len(final_cols_order))}{len(full_results_df) + 1}"
    tab = Table(displayName="ScoringClustersTable", ref=table_ref)
    style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                          showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    tab.tableStyleInfo = style
    ws.add_table(tab)
    
    wb.save(output_excel_path)
    print(f"✅ Excel Table created successfully")
    print(f"   📌 Table name: ScoringClustersTable")
    print(f"   📍 Selection_Status column: Click dropdown to filter SELECT or ALL")
except Exception as e:
    print(f"⚠️ Excel Table creation note: {e}")
    print(f"   Data is still present - manually add table via Data → Format as Table")

print(f"\n🎉 Analysis Complete!")
print(f"📊 Results saved to: {output_excel_path}")
print(f"📈 PCA plot saved to: {png_path}")

print(f"\n📋 Quick Preview - Top 5 companies by Additive Rank:")
preview_cols = ['Rank_Additive', 'Rank_Multiplicative', 'Ticker', 'Name', 'Selection_Status', 'Cluster_Name']
print(full_results_df.head(5)[preview_cols].to_string(index=False))

print(f"\n📋 Quick Preview - Top 5 companies by Multiplicative Rank:")
print(full_results_df.sort_values('Rank_Multiplicative').head(5)[preview_cols].to_string(index=False))

log_file_path = os.path.join(output_folder, "analysis_log.txt")
with open(log_file_path, 'a') as log_file:
    log_file.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Analysis completed: {output_excel_name}\n")
print(f"\n📝 Analysis logged in: {log_file_path}")

✅ Input file found: koyfin_Junior Miners.csv
📅 File date: 2025-11-04 10:24:46
Input file: G:\My Drive\INPUTS\EXCEL\Miners\koyfin_Junior Miners.csv
Output file will be saved to: G:\My Drive\OUTPUTS\EXCEL\Miner\Junior_Miner_Analysis_V6_20251104_120600.xlsx
📅 Timestamp: 20251104_120600

✅ SELECT file found: koyfin_Select Miners.csv
   📌 Loaded 18 companies for comparison (via ISIN matching)
   🔍 Sample: UURAF, TMQ, ARA, HAS, NB...

📈 Loaded 498 companies with 41 columns
   📊 Selection Status: 17/18 SELECT companies found in data

🔍 Applying Data Quality Screen...
   - 84 companies excluded for missing critical data.
⚠️ Applying Collapsed Market Cap Screen...
   - 33 companies flagged for collapsed market cap.
   ✅ 381 companies remaining for full analysis.

🛠️ Engineering and transforming features for scoring consistency...

⚙️ Scoring Model: 7 factors defined.
🔢 Processing 12 KPIs for scoring...

⚖️ Normalizing factor scores and calculating Total Scores...
✅ Scoring & Normalization compl

C:\Users\ashle\AppData\Local\Temp\ipykernel_38872\3974996651.py:200: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  score_df[col].fillna(median_val, inplace=True)
C:\Users\ashle\AppData\Local\Temp\ipykernel_38872\3974996651.py:200: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

📊 PCA plot saved to: G:\My Drive\OUTPUTS\EXCEL\Miner\Junior_Miner_Analysis_V6_20251104_120600_PCA_k3.png

📝 Generating Tier & Cluster Profile Analysis tables...
📈 Generating Combined Tier-Cluster 3M Return Analysis...

📊 Generating Excel Output...
✅ Excel Table created successfully
   📌 Table name: ScoringClustersTable
   📍 Selection_Status column: Click dropdown to filter SELECT or ALL

🎉 Analysis Complete!
📊 Results saved to: G:\My Drive\OUTPUTS\EXCEL\Miner\Junior_Miner_Analysis_V6_20251104_120600.xlsx
📈 PCA plot saved to: G:\My Drive\OUTPUTS\EXCEL\Miner\Junior_Miner_Analysis_V6_20251104_120600_PCA_k3.png

📋 Quick Preview - Top 5 companies by Additive Rank:
 Rank_Additive  Rank_Multiplicative Ticker                                Name Selection_Status       Cluster_Name
             1                    3    APM        Andean Precious Metals Corp.              ALL Stalwart Survivors
             2                    1    MMY             Monument Mining Limited              ALL Stalwa